In [5]:
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import StreamingResponse
from pydantic import BaseModel
import asyncio
import json

# Import your modified run_finance_agent function
from your_finance_agent_module import run_finance_agent

app = FastAPI()

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class QueryInput(BaseModel):
    query: str

async def event_generator(query):
    async for state in run_finance_agent(query):
        yield f"data: {json.dumps(state)}\n\n"
    yield "data: [DONE]\n\n"

@app.post("/query")
async def process_query(query_input: QueryInput):
    return StreamingResponse(event_generator(query_input.query), media_type="text/event-stream")

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Finance Agent Query Interface</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            max-width: 800px;
            margin: 0 auto;
            padding: 20px;
        }
        #query-form {
            display: flex;
            margin-bottom: 20px;
        }
        #query-input {
            flex-grow: 1;
            padding: 10px;
            font-size: 16px;
        }
        #submit-button {
            padding: 10px 20px;
            font-size: 16px;
            background-color: #4CAF50;
            color: white;
            border: none;
            cursor: pointer;
        }
        #result {
            border: 1px solid #ddd;
            padding: 20px;
            min-height: 100px;
        }
    </style>
</head>
<body>
    <h1>Finance Agent Query Interface</h1>
    <form id="query-form">
        <input type="text" id="query-input" placeholder="Enter your query here" required>
        <button type="submit" id="submit-button">Submit</button>
    </form>
    <div id="result"></div>

    <script>
        document.getElementById('query-form').addEventListener('submit', async (e) => {
            e.preventDefault();
            const query = document.getElementById('query-input').value;
            const resultDiv = document.getElementById('result');
            resultDiv.innerHTML = 'Processing...';

            try {
                const response = await fetch('http://localhost:8000/query', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json',
                    },
                    body: JSON.stringify({ query: query }),
                });

                if (!response.ok) {
                    throw new Error(`HTTP error! status: ${response.status}`);
                }

                const data = await response.json();
                resultDiv.innerHTML = `<h2>Result:</h2><p>${data.result}</p>`;
            } catch (error) {
                resultDiv.innerHTML = `<p>Error: ${error.message}</p>`;
            }
        });
    </script>
</body>
</html>